# Universidade Tecnológica Federal do Paraná

## Programa de Pós-Graduação em Computação Aplicada

### Ciência de Dados 2 - 2021/1
### Equipe Evolution:
### Leila Fabiola Ferreira
### Mateus Cichelero da Silva

## Atividade 2 - Análise exploratória: Arquivo A - Extração de dados

Este primeiro script tem por objetivo realizar as etapas referentes à extração de dados das mensagens enviadas durante a pandemia para o bot de atendimento Laura - PA Digital.

Trata-se de um bot onde as pessoas podem enviar suas dúvidas sobre o coronavírus/pandemia e serem atendidas de forma automática. Um exemplo de aplicação dele pode ser conferido no site da secretaria de saúde de Curitiba, basta clicar no widget localizado no canto inferior direito da página ("Perguntas sobre o Coronavírus"): https://saude.curitiba.pr.gov.br/

A aplicação é proprietária, e os dados originais estão localizados em uma base PostgreSQL. O script abaixo trabalha na extração de dados do período de 25/março 2020 até 20/março 2021, totalizando 1,26 milhões de mensagens recebidas em 1,4GB de dados. Para evitar problemas de memória com esses dados brutos, os arquivos foram extraídos por etapa através de paginação, processando 10 mil mensagens por página.

O próximo arquivo B_laura_data_processing se encarregará de formatar essa grande massa de dados em um único arquivo mais enxuto.

In [1]:
import psycopg2
import pandas as pd
from datetime import datetime

In [2]:
try:
    connection = psycopg2.connect(user="ALTERAR_CREDENCIAIS",
                                  password="ALTERAR_CREDENCIAIS",
                                  host="ALTERAR_CREDENCIAIS",
                                  port="ALTERAR_CREDENCIAIS",
                                  database="ALTERAR_CREDENCIAIS")
    cursor = connection.cursor()
    postgreSQL_select_Query = """SELECT id, "timestamp", data FROM events 
                                 WHERE type_name = 'user' AND "timestamp" < 1617246000 
                                 ORDER BY id
                                 LIMIT %s
                                 OFFSET %s;"""
    
    # Limitando o número de mensagens por página em 10mil:
    limit = 10000
    offset = 10000
    max_paginas = 500
    
    for index_pagina in range(max_paginas):
        print('Página '+str(index_pagina)+' #################')
        cursor.execute(postgreSQL_select_Query,(limit,offset*index_pagina))
        events_records = cursor.fetchall()
        df = pd.DataFrame(events_records,columns =['id', 'date_timestamp', 'data'],dtype='str')
        print(datetime.fromtimestamp(float(df.iloc[-1]['date_timestamp'])))
        df.to_csv('extraction_data/'+str(index_pagina)+'_data.csv', header=True, index=False)


except (Exception, psycopg2.Error) as error:
    print("Error while fetching data from PostgreSQL", error)

finally:
    # closing database connection.
    if connection:
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")

Página 0 #################
2020-04-21 09:45:29.881780
Página 1 #################
2020-04-28 15:07:18.174580
Página 2 #################
2020-05-02 11:40:50.976580
Página 3 #################
2020-05-06 16:40:42.233660
Página 4 #################
2020-05-11 20:09:57.275690
Página 5 #################
2020-05-15 18:14:42.992240
Página 6 #################
2020-05-20 22:42:25.653170
Página 7 #################
2020-05-26 13:30:04.124910
Página 8 #################
2020-05-29 08:44:08.162340
Página 9 #################
2020-06-01 19:14:12.276920
Página 10 #################
2020-06-06 08:44:13.740320
Página 11 #################
2020-06-10 11:42:32.479060
Página 12 #################
2020-06-13 19:43:54.168770
Página 13 #################
2020-06-15 13:06:44.652670
Página 14 #################
2020-06-18 08:16:12.283800
Página 15 #################
2020-06-21 14:54:13.543960
Página 16 #################
2020-06-24 15:01:07.710260
Página 17 #################
2020-06-28 08:00:14.925690
Página 18 ##########

KeyboardInterrupt: 

Obs: Após a extração de 126 páginas (1,26 Milhão de mensagens) o script foi interrompido manualmente por se considerar o suficiente para as análises.